In [2]:
import numpy as np
import pygame as pg
import matplotlib.pyplot as plt
import random

xs = np.linspace(-1, 1, 15)
ys = np.linspace(-1, 1, 15)
zs = np.linspace(-1, 1, 15)


rot_x = 0
rot_y = 0
rot_z = 0

rotv_x = 0
rotv_y = 0
rotv_z = 0


vectors = np.array([[0, 0, 0]])
for x in xs:
    for y in ys:
        for z in zs:
            vectors = np.append(vectors, np.array([[x, y, z]]), axis=0)
            #vectors = np.append(vectors, np.array([[x, np.sin(x**2 + z**2)*0.2, z]]), axis=0)


running = True
pg.init()
width, height = 800, 800
bg_color = (0, 0, 0)
screen = pg.display.set_mode((width, height))
clock = pg.time.Clock()
rot = 0
show_transformation = False

def rotate_z(vec, rotation):
    rotmat = np.array([[np.cos(rotation), -np.sin(rotation), 0],
    [np.sin(rotation), np.cos(rotation), 0],
    [0, 0, 1]])
    return np.matmul(rotmat, vec)

def rotate_y(vec, rotation):
    rotmat = np.array(
            [[np.cos(rotation), 0, np.sin(rotation)],
            [0, 1, 0],
            [-np.sin(rotation), 0, np.cos(rotation)]])
    return np.matmul(rotmat, vec)

def rotate_x(vec, rotation):
    rotmat = np.array([
            [1, 0, 0],
            [0, np.cos(rotation), -np.sin(rotation)],
            [0, np.sin(rotation), np.cos(rotation)]])
    return np.matmul(rotmat, vec)

projection_matrix = np.array([[1,0,0],[0,1,0]])
scale = 0.5
scale_matrix = np.array([[scale, 0, 0], [0, scale, 0], [0, 0, scale]])
mouse_down = False

mouse_x = 0
mouse_y = 0

while running:
    dx, dy = pg.mouse.get_rel()
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        if event.type == pg.MOUSEBUTTONDOWN:
            mouse_down = True
        if event.type == pg.MOUSEBUTTONUP:
            mouse_down = False
        if event.type == pg.MOUSEMOTION:
            mouse_x, mouse_y = event.pos
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_ESCAPE:
                running = False
            if event.key == pg.K_m:
                show_transformation = not show_transformation

    if mouse_down:
        rot_y += dx*0.005
        rot_x += dy*0.005
        rotv_y += dx*0.001
        rotv_x += dy*0.001    
    screen.fill(bg_color)
    ball_index = 0
    rot_x += rotv_x + 0.001
    rot_y += rotv_y + 0.001
    rot_z += rotv_z + 0.001
    rotv_x *= 0.9
    rotv_y *= 0.9
    rotv_z *= 0.9


    for point in vectors:
        line_size = 0
        ball_color = (point+1)*125
        
        scaled_point = np.matmul(scale_matrix, point)
        scaled_point = np.array([scaled_point]).T
        saver = np.array(scaled_point)
        p = np.matmul(scaled_point, scaled_point.T)
        scaled_point = np.matmul(p, scaled_point)
        rpoint = rotate_y(scaled_point, rot_y)
        rpoint = rotate_x(rpoint, rot_x)
        rpoint = rotate_z(rpoint, rot_z)
        
        z = 2/(2+rpoint[2][0])
        ppoint = np.matmul(projection_matrix, rpoint)
        x, y = (ppoint[0][0])*width/2 + width/2, (ppoint[1][0])*height/-2 + height/2
        x*=z
        y*=z
        ##here we find distancio
        distanceroni = ((mouse_x-x)**2 + (mouse_y-y)**2)**0.5
        distanceroni /= np.linalg.norm(np.array([width*2, height*2]))
        pg.draw.circle(screen, ball_color*((1-distanceroni)**4), (x, y), 2 + int(4*((1-distanceroni)**4)), line_size)
        if show_transformation:
            po = np.matmul(projection_matrix, saver)
            x0, y0 = (po[0][0])*width/2 + width/2, (po[1][0])*height/-2 + height/2
            pg.draw.line(screen, ball_color, (x0, y0), (x, y), 1)
        ball_index+= 1
    pg.display.flip()
    clock.tick(30)
pg.quit()


In [1]:
import pygame as pg
import numpy as np

vectors = np.array([[0, 0]]).T
for x in np.linspace(-1, 1, 20):
    for y in np.linspace(-1, 1, 20):
        vectors = np.append(np.array([[x, y]]).T, vectors, axis=1)
width, height = 500, 500
pg.init()
screen = pg.display.set_mode((width, height), pg.RESIZABLE)
clock = pg.time.Clock()
running = True

while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_ESCAPE:
                running = False
    width, height = screen.get_size()
    screen.fill((0, 0, 0))
    proj = np.matmul(np.array([[0.75, 0], [0, 0.75]]), vectors)
    #proj += np.ones_like(proj)
    #final_projections = np.matmul(np.array([[width/2, 0], [0, height/2]]), proj)
    for projection_point in proj.T:
        pp = np.array([projection_point]).T
        #pp = np.matmul(np.array([[1/width, 0],[0, 1/height]]), pp)
        p = np.matmul(pp, pp.T)
        pp = np.matmul(p, pp)
        pp += np.ones_like(pp)
        pp = np.matmul(np.array([[0.5, 0],[0,0.5]]), pp)
        pp = np.matmul(np.array([[width, 0], [0, height]]), pp)
        print(pp)
        pg.draw.circle(screen, (255, 255, 255), (pp[0][0], pp[1][0]), 1, 0)
    pg.display.flip()
    clock.tick(30)
pg.quit()



9407]
 [248.00102967]]
[[250.76883474]
 [244.6181568 ]]
[[251.26088898]
 [238.6519992 ]]
[[251.87595677]
 [229.36447551]]
[[252.61403813]
 [216.01750437]]
[[253.47513304]
 [197.87300445]]
[[254.45924151]
 [174.19289437]]
[[255.56636354]
 [144.2390928 ]]
[[244.43363646]
 [355.7609072 ]]
[[245.54075849]
 [325.80710563]]
[[246.52486696]
 [302.12699555]]
[[247.38596187]
 [283.98249563]]
[[248.12404323]
 [270.63552449]]
[[248.73911102]
 [261.3480008 ]]
[[249.23116526]
 [255.3818432 ]]
[[249.60020593]
 [251.99897033]]
[[249.84623305]
 [250.46130085]]
[[249.96924661]
 [250.03075339]]
[[249.96924661]
 [249.96924661]]
[[249.84623305]
 [249.53869915]]
[[249.60020593]
 [248.00102967]]
[[249.23116526]
 [244.6181568 ]]
[[248.73911102]
 [238.6519992 ]]
[[248.12404323]
 [229.36447551]]
[[247.38596187]
 [216.01750437]]
[[246.52486696]
 [197.87300445]]
[[245.54075849]
 [174.19289437]]
[[244.43363646]
 [144.2390928 ]]
[[232.93186871]
 [358.09816482]]
[[236.2532348 ]
 [327.89833613]]
[[239.20556021]
 [30

In [4]:
import numpy as np
import scipy as sp
from scipy import linalg as lg

a = np.array([[6, -1, -1], [1, 1, 1 ], [2, -1, 1]])
pl, u = lg.lu(a, permute_l=True)
print(pl)
print("----")
print(u)

[[ 1.          0.          0.        ]
 [ 0.16666667  1.          0.        ]
 [ 0.33333333 -0.57142857  1.        ]]
----
[[ 6.         -1.         -1.        ]
 [ 0.          1.16666667  1.16666667]
 [ 0.          0.          2.        ]]


In [11]:
#Stole code from https://gist.github.com/j9ac9k/6b5cd12aa9d2e5aa861f942b786293b4
#dat guy a king :D
def gauss(A):
    m = len(A)
    assert all([len(row) == m + 1 for row in A[1:]]), "Matrix rows have non-uniform length"
    n = m + 1
    
    for k in range(m):
        pivots = [abs(A[i][k]) for i in range(k, m)]
        i_max = pivots.index(max(pivots)) + k
        
        # Check for singular matrix
        assert A[i_max][k] != 0, "Matrix is singular!"
        
        # Swap rows
        A[k], A[i_max] = A[i_max], A[k]

        
        for i in range(k + 1, m):
            f = A[i][k] / A[k][k]
            for j in range(k + 1, n):
                A[i][j] -= A[k][j] * f

            # Fill lower triangular matrix with zeros:
            A[i][k] = 0
    
    # Solve equation Ax=b for an upper triangular matrix A         
    x = []
    for i in range(m - 1, -1, -1):
        x.insert(0, A[i][m] / A[i][i])
        for k in range(i - 1, -1, -1):
            A[k][m] -= A[k][i] * x[0]
    return x

print(gauss(
    [[6, -1, -1, 4],
    [1, 1, 10, -6],
    [2, -1, 1, -2]]
    ))


[0.9999999999999999, 2.999999999999999, -1.0]


In [2]:
coma = [[1, 0, 0, 0, 0, 0, 0],
        [-1, 2, -1, 0, 0, 0, 1],
        [0, -1, 2, -1, 0, 0, 2],
        [0, 0, -1, 2, -1, 0, 3],
        [0, 0, 0, -1, 2, -1, 4],
        [0, 0, 0, 0, 0, 1, 0]]

## Gauss aus der Folie

Hier der Code aus der Folie:


In [4]:
def rowMod0(M, i, j, x):
    M[i]= [a+x*b for a, b in zip(M[i], M[j])]

#Dis jus makes le MAtrix to upper or downer
def rowEchelon0(M):
    row, col = 0, 0
    rows, cols = len(M), len(M[0])
    while row < rows and col < cols:
        if M[row][col] == 0:
            for r in range(row +1, rows):
                if not M[r][col] == 0:
                    rowMod0(M, row, r, 1)
                    break
        if M[row][col] == 0:
            col += 1
            continue
        pivot = M[row][col]
        for r in range(row +1, rows):
            if not M[r][col] == 0:
                rowMod0(M, r, row, -M[r][col]/pivot)
        row += 1
        col += 1
    return M

print(rowEchelon0(coma))

[[1, 0, 0, 0, 0, 0, 0], [0.0, 2.0, -1.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.5, -1.0, 0.0, 0.0, 2.5], [0.0, 0.0, 0.0, 1.3333333333333335, -1.0, 0.0, 4.666666666666666], [0.0, 0.0, 0.0, 0.0, 1.25, -1.0, 7.499999999999999], [0, 0, 0, 0, 0, 1, 0]]
